# Blue Brain BioExplorer

![](../bioexplorer_banner.png)

## Simplest script to create a visualization of an enzyme reaction

In [ ]:
from bioexplorer import BioExplorer, Protein, EnzymeReaction, MolecularSystemAnimationParams, \
                        Volume, Vector3, Quaternion
import os

be = BioExplorer('localhost:5000')
core = be.core_api()
status = be.reset_scene()

In [ ]:
representation=be.protein_representation.ATOMS_AND_STICKS
atom_radius_multiplier=1.0
draft = True
generate_movie = False

# Resources
resource_folder = os.path.abspath('../../tests/test_files')
pdb_folder = os.path.join(resource_folder, 'pdb')
metabolites_folder = os.path.join(pdb_folder, 'metabolites')
enzymes_folder = os.path.join(pdb_folder, 'enzymes')
mesh_folder = os.path.join(resource_folder, 'obj')

pdb_glucose = os.path.join(metabolites_folder, '5793.pdb')
pdb_atp = os.path.join(metabolites_folder, '59.pdb')
pdb_g6p = os.path.join(metabolites_folder, '439284.pdb')
pdb_adp = os.path.join(metabolites_folder, '6022.pdb')
pdb_hexokinase = os.path.join(enzymes_folder, '1bdg.pdb')

be.set_general_settings(logging_level=3, mesh_folder=mesh_folder)

scene_size = Vector3(200.0, 200.0, 200.0)
nb_occurences = 200
seed = 5

In [ ]:
def add_molecule(
        seed, position_seed, name, pdb_file, nb_occurence,
        representation=be.protein_representation.ATOMS_AND_STICKS, rotation=Quaternion()):
    protein = Protein(
        name=name,
        source=pdb_file,
        load_bonds=True, load_hydrogen=True, load_non_polymer_chemicals=True,
        occurrences=nb_occurence, animation_params=MolecularSystemAnimationParams(seed, position_seed, 25.0, seed, 0.025),
        rotation=rotation
    )
    volume = Volume(
        name=name + 's',
        shape=be.assembly_shape.CUBE, shape_params=scene_size,
        protein=protein)
    be.add_volume(
        volume=volume,
        atom_radius_multiplier=atom_radius_multiplier, representation=representation)
    return protein

# Substrates
substrates = list()
substrates.append(
    add_molecule(seed, 6, 'Glucose', pdb_glucose, nb_occurences))
substrates.append(
    add_molecule(seed, 7, 'ATP', pdb_atp, nb_occurences))

# Product
products = list()
products.append(
    add_molecule(seed, 8, 'G6P', pdb_g6p, nb_occurences))
products.append(
    add_molecule(seed, 9, 'ADP', pdb_adp, nb_occurences))

# Hexokinase
enzyme = add_molecule(
    seed, 0, 'Hexokinase', pdb_hexokinase, nb_occurences, 
    be.protein_representation.ATOMS_AND_STICKS, Quaternion(0.707, 0.707, 0.0, 0.0)
)

# Enzyme reaction
enzyme_reaction = EnzymeReaction(
    assembly_name='Hexokinase',
    name='Hexokinase',
    enzyme=enzyme,
    substrates=substrates,
    products=products)

# Add the reaction to the scene
status = be.add_enzyme_reaction(enzyme_reaction)

## Rendering settings

In [ ]:
status = be.core_api().set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='advanced',
    samples_per_pixel=1, subsampling=4, max_accum_frames=16)
params = status = be.core_api().AdvancedRendererParams()
params.gi_samples = 2
params.gi_strength = 0.2
params.gi_ray_length = 5
params.shadow_intensity = 1.0
params.soft_shadow_strength = 1.0
params.fog_start = 1500
params.fog_thickness = 1500
params.max_ray_depth = 1
status = be.core_api().set_renderer_params(params)

## Materials

In [ ]:
status = be.set_protein_color_scheme(
    assembly_name='ATPs', name='ATP',
    color_scheme=be.protein_color_scheme.ATOMS,
    palette_name='Blues', palette_size=2)

status = be.set_protein_color_scheme(
    assembly_name='Glucoses', name='Glucose',
    color_scheme=be.protein_color_scheme.ATOMS,
    palette_name='Blues', palette_size=2)


status = be.set_protein_color_scheme(
    assembly_name='ADPs', name='ADP',
    color_scheme=be.protein_color_scheme.ATOMS,
    palette_name='Set1', palette_size=3)

status = be.set_protein_color_scheme(
    assembly_name='G6Ps', name='G6P',
    color_scheme=be.protein_color_scheme.ATOMS,
    palette_name='Set1', palette_size=3)

status = be.set_protein_color_scheme(
    assembly_name='Hexokinases', name='Hexokinase',
    color_scheme=be.protein_color_scheme.CHAINS,
    palette_name='Reds', palette_size=2)
be.core_api().set_renderer()

## Movie

In [ ]:
from bioexplorer import MovieMaker
mm = MovieMaker(be)

In [ ]:
if draft:
    instance_id = 2
    key_frames = [
        {
            'apertureRadius': 0.0,
            'direction': [-0.4228059199334163, 0.13956182704765893, -0.8954092084071824],
            'focalDistance': 1000000.0,
            'origin': [-76.94388312721057, 27.008762696107365, 101.90818416814368],
            'up': [0.03825535069708487, 0.9899381257456357, 0.13623155045091803]
        }, 
        {
            'apertureRadius': 0.0,
            'direction': [-0.2931756929393873, -0.19137884221190335, -0.9367081465553406],
            'focalDistance': 1000000.0,
            'origin': [-80.78751362339919, 32.810933007328735, 98.7370846157903],
            'up': [-0.04288359642471887, 0.9814064493156495, -0.18708922577030282]
        }
    ]
else:
    instance_id = 25
    key_frames = [
        {
            'apertureRadius': 0.0,
            'direction': [-0.813281188551245, -0.09740833983733163, -0.5736595886750337],
            'focalDistance': 1000000.0,
            'origin': [11.06562396175978, -42.45183472913227, 71.07319024762178],
            'up': [-0.26548353789130863, 0.9394084115205614, 0.2168647677081957]
        }, 
        {
            'apertureRadius': 0.0,
            'direction': [-0.06265926430929034, -0.21184739821321946, -0.975292005743156],
            'focalDistance': 1000000.0,
            'origin': [-3.900908980540925, -40.62438454349056, 76.12253643837245],
            'up': [0.09475544149237422, 0.9715352310065103, -0.21711909455575734]
        }
    ]    

In [ ]:
from tqdm import tqdm

nb_frames = 360
output_folder = '/tmp'
if draft:
    image_size=[960, 540]
else:
    image_size=[3840, 2160]

mm.build_camera_path(key_frames, nb_frames, 1)

if generate_movie:
    for frame in tqdm(range(nb_frames)):
        mm.set_current_frame(frame)
        for instance in range(nb_occurences):
            progress = float(frame + instance) / float(nb_frames)
            be.set_enzyme_reaction_progress(
                enzyme_reaction, instance_id=instance, progress=progress)
        mm.create_snapshot(
            renderer='advanced',
            path=os.path.join(output_folder, 'advanced'), base_name='%05d' % frame,
            size=image_size, samples_per_pixel=64)
        mm.create_snapshot(
            renderer='depth',
            path=os.path.join(output_folder, 'depth'), base_name='%05d' % frame,
            size=image_size, samples_per_pixel=1)
else:
    be.core_api().set_renderer(
        current='basic',
        samples_per_pixel=1, subsampling=1, max_accum_frames=1)
    model_id = core.scene.models[len(core.scene.models)-1]['id']
    import time
    for frame in tqdm(range(nb_frames)):
        mm.set_current_frame(frame)
        progress = float(frame) / float(nb_frames)
        for instance in range(nb_occurences):
            progress = float(frame + instance) / float(nb_frames)
            be.set_enzyme_reaction_progress(
                enzyme_reaction, instance_id=instance, progress=progress)
        be.core_api().set_renderer()
        time.sleep(0.1)